In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

import flwr as fl

ModuleNotFoundError: No module named 'flwr'

In [12]:
def get_datasets(dataset, class_imbalance = False, sample_imbalance = False):
    if dataset == 'banana':
        datasets =  ["/home/swier/Documents/afstuderen/nnTest/v6_simpleNN_py/local/banana/banana_dataset_client" + str(i) + ".csv" for i in range(10)]
    elif dataset == 'MNIST':
        datasets= ["/home/swier/Documents/afstuderen/nnTest/v6_simpleNN_py/local/MNIST/MNIST_dataset_client" + str(i) + ".csv" for i in range(10)]
    elif dataset == 'MNIST_2class':
        if class_imbalance:
            datasets = ["/home/swier/Documents/afstuderen/datasets/MNIST_2Class_class_imbalance/MNIST_2Class_class_imbalance_client" + str(i) + ".csv" for i in range(10)]
        elif sample_imbalance:
            datasets =["/home/swier/Documents/afstuderen/datasets/MNIST_2Class_Sample_Imbalance/MNIST_2Class_sample_imbalance_client" + str(i) + ".csv" for i in range(10)]
        else:
            datasets= ["/home/swier/Documents/afstuderen/datasets/MNIST_2Class_IID/MNIST_2Class_IID_client" + str(i) + ".csv" for i in range(10)]
    elif dataset == 'MNIST_4class':
        if class_imbalance:
            datasets = ["/home/swier/Documents/afstuderen/datasets/4Class_class_imbalance/MNIST_4Class_class_imbalance_client" + str(i) + ".csv" for i in range(10)]
        elif sample_imbalance:
            datasets = ["/home/swier/Documents/afstuderen/datasets/4Class_sample_imbalance/MNIST_4Class_sample_imbalance_client" + str(i) + ".csv" for i in range(10)]
        else:
            datasets = ["/home/swier/Documents/afstuderen/datasets/4Class_IID/MNIST_4Class_IID_client" + str(i) + ".csv" for i in range(10)]
    elif dataset == "fashion_MNIST" :
        if class_imbalance:
            datasets = ["/home/swier/Documents/afstuderen/datasets/fashion_MNIST_ci/fashion_MNIST_superCI_client" + str(i) + ".csv" for i in range(10)]
        else:
            datasets = ["/home/swier/Documents/afstuderen/datasets/fashion_mnist/csv/fashion_MNIST_dataset_client" + str(i) + ".csv" for i in range(10)]
    elif dataset == "A2_PCA" :
        if class_imbalance:
            datasets = ["/home/swier/Documents/afstuderen/datasets/RMA/A2/PCA/class imbalance/A2_PCA_client"+ str(i) + ".csv" for i in range(10)]
        elif sample_imbalance:
            datasets = ["/home/swier/Documents/afstuderen/datasets/RMA/A2/PCA/sample imbalance/AML_A2_PCA_client"+ str(i) + ".csv" for i in range(10)]
        else:
            datasets = ["/home/swier/Documents/afstuderen/datasets/RMA/A2/PCA/IID/AML_A2_PCA_client"+ str(i) + ".csv" for i in range(10)]
    elif dataset == "A2_raw" : 
        datasets = ["/home/swier/Documents/afstuderen/datasets/RMA/A2/AML_A2_client"+ str(i) + ".csv" for i in range(10)]
    elif dataset == "3node" :
        if class_imbalance: 
            datasets = ["/home/swier/Documents/afstuderen/datasets/RMA/A" + str(i) + "/3node_PCA_A" + str(i) + ".csv" for i in range(1,4)]
        else :
            datasets = ["/home/swier/Documents/afstuderen/datasets/RMA/A" + str(i) + "/3node_PCA_balanced_A" + str(i) + ".csv" for i in range(1,4)]
    elif dataset == "2node" : 
        datasets = ["/home/swier/Documents/afstuderen/datasets/RMA/A" + str(i) + "/2node_PCA_A" + str(i) + ".csv" for i in range(1,3)]
    else :
        raise(ValueError("unknown dataset"))
    
    return datasets


In [32]:
def get_data(dataset, client_id, class_imbalance=False, sample_imbalance=False, model_choice="FNN"):
    datasets_paths= get_datasets(dataset,class_imbalance, sample_imbalance)

    dataset_path = datasets_paths[client_id]
    
    df = pd.read_csv(path)
    
    X_train_arr = df.loc[df['test/train'] == 'train'].drop(columns = ['test/train', 'label']).values
    y_train_arr = df.loc[df['test/train'] == 'train']['label'].values
    X_test_arr = df.loc[df['test/train'] == 'test'].drop(columns = ["test/train", "label"]).values
    y_test_arr = df.loc[df['test/train'] == 'test']['label'].values
    
    X_train = torch.as_tensor(X_train_arr, dtype=torch.double)
    y_train = torch.as_tensor(y_train_arr, dtype=torch.int64)
    X_test = torch.as_tensor(X_test_arr, dtype=torch.double)
    y_test = torch.as_tensor(y_test_arr, dtype=torch.int64)
    
    if model_choice == "CNN":
        reshape_size = int(math.sqrt(X_test.shape[1]))
        X_test = X_test.reshape(X_test.shape[0], 1, reshape_size, reshape_size)
        X_train = X_train.reshape(X_train.shape[0], 1, reshape_size, reshape_size)
    
    return X_train, y_train, X_test, y_test    
    
  

In [5]:
class MNIST_2class_FNN(nn.Module):
    def __init__(self):
        super(MNIST_2class_FNN, self).__init__()
        self.lin1 = nn.Linear(28*28,100)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(100,2)
        
    def forward(self, input):
        y1 = self.lin1(input)
        y2 = self.relu(y1)
        return(self.lin2(y2))
    
    def train(self,X, Y, lr):
        optimizer = optim.SGD(self.parameters(), lr=lr)
        optimizer.zero_grad()
        criterion = nn.CrossEntropyLoss()
        
        out = self.forward(X)
        loss = criterion(out, Y)
        loss.backward()
        optimizer.step()
        
    def test(self, X, Y):
        correct = 0
        with torch.no_grad():
            out = self.forward(X)
            _, pred = torch.max(out.data, 1)
            correct += (pred==Y).sum().item()
        return (correct / X.size()[0])

In [24]:
def run(rounds, runs, batch_amount, lr,  ci = False, si = False):

    results = np.zeros((runs, rounds))
    X_train, y_train, X_test, y_test = get_data("MNIST_2class", ci, si, "FNN")
    for run in range(runs):
        torch.manual_seed(run)
        net = MNIST_2class_FNN()
        net = net.double()
        optimizer = optim.SGD(net.parameters(), lr = lr)
        criterion = nn.CrossEntropyLoss()

        for roun in range(rounds):
            net.train(X_train, y_train,  lr)
            results[run, roun] = net.test(X_test, y_test)
    return results

In [36]:
class FlClient(fl.client.NumPyClient):
    def __init__(self, client_id, lr):
        super (FlClient, self).__init__()
        self.net = MNIST_2class_FNN()
        self.X_train, self.y_train, self.X_test, self.y_test = get_data("MNIST_2class", client_id)
        self.lr = lr
        
    def get_parameters(self):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]
    
    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)  

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.net.train(X_train, y_train, self.lr)
        return self.get_parameters(), num_examples["trainset"], {}

In [37]:
fl.server.start_server(config={"num_rounds": 3})

INFO flower 2022-02-28 09:27:27,709 | app.py:77 | Flower server running (insecure, 3 rounds)
INFO flower 2022-02-28 09:27:27,710 | server.py:118 | Initializing global parameters
INFO flower 2022-02-28 09:27:27,710 | server.py:304 | Requesting initial parameters from one random client


KeyboardInterrupt: 